# StreamVC Training on Google Colab

このノートブックでは、Google Colab上でStreamVCの学習を実行します。

## 前提条件
- Google Driveに`streamVC/data/`がアップロード済み
- GitHubリポジトリが公開されている

## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. リポジトリのクローン

In [ ]:
# GitHubリポジトリのURL（あなたのリポジトリに変更してください）
REPO_URL = "https://github.com/SteamVC/StreamVC_model.git"

!git clone {REPO_URL}
%cd StreamVC_model

## 4. 依存パッケージのインストール

In [ ]:
!pip install -e . -q

## 5. キャッシュ生成（初回のみ / 修正後の再生成時）

Google Drive の生データから、修正版 whitening を適用したキャッシュを生成して Google Drive に保存します。

**既にキャッシュがある場合はスキップしてください。**

In [ ]:
import subprocess
from pathlib import Path

# Google Drive のキャッシュパス
GDRIVE_CACHE = "/content/drive/MyDrive/streamVC/data/cache/libri_tts"

# キャッシュが既に存在するかチェック
def count_cache_files(path):
    try:
        result = subprocess.run(
            f"ls -1 '{path}/train' 2>/dev/null | wc -l",
            shell=True, capture_output=True, text=True
        )
        return int(result.stdout.strip())
    except:
        return 0

existing_count = count_cache_files(GDRIVE_CACHE)
print(f"既存キャッシュ: {existing_count} files")

if existing_count > 30000:
    print("✓ キャッシュは既に存在します")
    print("  修正版で再生成する場合は、次のセルを実行してください")
else:
    print("⚠ キャッシュが不足しています")
    print("  次のセルでキャッシュを生成してください")

In [ ]:
# Whitening の漏れをチェック
!python scripts/check_whitening_leak.py

print("\n期待される結果:")
print("  F0 各サンプル平均の分散: ≈ 0.000 (修正前: 0.339)")
print("  Energy 各サンプル平均の分散: ≈ 0.000 (修正前: 0.431)")

### 5.2. Whitening 検証

生成したキャッシュで per-utterance whitening が正しく機能しているか確認します。

In [ ]:
# キャッシュ生成（Google Drive に直接保存）
print("=" * 60)
print("キャッシュ生成開始（修正版 per-utterance whitening）")
print("=" * 60)

# Config を使ってキャッシュ生成
!python scripts/preprocess.py \
    --config configs/colab_cache_gen.yaml \
    --device cuda \
    --batch-size 32

print("\n" + "=" * 60)
print("キャッシュ生成完了")
print("=" * 60)

# 生成されたファイル数を確認
train_count = count_cache_files(GDRIVE_CACHE)
print(f"生成されたキャッシュ: {train_count} files")

if train_count > 30000:
    print("✓ キャッシュ生成成功")
else:
    print("⚠ キャッシュが不足しています（エラーが発生した可能性）")

### 5.1. キャッシュ生成実行

**注意**: これには 30-60分 かかります。GPU を使用します。

## 6. Checkpointディレクトリの設定

In [ ]:
# Checkpoint保存先をGoogle Driveに設定（正しいパス）
CHECKPOINT_DIR = "/content/drive/MyDrive/streamVC/checkpoints"

!mkdir -p {CHECKPOINT_DIR}

print(f"Checkpoints will be saved to: {CHECKPOINT_DIR}")

## 7. 設定ファイルの準備（Colab用）

In [ ]:
# Colab用の設定ファイルを確認
!cat configs/colab_gpu_training.yaml

## 7.5. DataLoader動作確認（学習前に実行推奨）

学習がハングする問題を診断するため、DataLoaderの動作をテストします。

In [ ]:
import time
import torch
import sys
from pathlib import Path

# 作業ディレクトリがStreamVC_modelであることを確認
import os
if not os.path.exists("configs/colab_gpu_training.yaml"):
    print("エラー: StreamVC_modelディレクトリにいません")
    print("セル6 (リポジトリのクローン) を実行してください")
    raise RuntimeError("Wrong directory")

# srcディレクトリをPythonパスに追加
sys.path.insert(0, str(Path.cwd() / "src"))

from streamvc.config import load_config
from streamvc.data.dataset import build_dataloader

print("=" * 60)
print("DataLoader動作確認スクリプト")
print("=" * 60)

# Config読み込み
config_path = "configs/colab_gpu_training.yaml"
print(f"\n1. Config読み込み: {config_path}")
cfg = load_config(config_path)
print(f"   ✓ Batch size: {cfg.training.batch_size}")
print(f"   ✓ Sample rate: {cfg.data.sample_rate}")

# DataLoader構築（時間計測）
print("\n2. DataLoader構築中...")
start = time.time()
try:
    train_loader = build_dataloader(cfg, split="train", batch_size=cfg.training.batch_size)
    build_time = time.time() - start
    print(f"   ✓ 構築完了: {build_time:.2f}秒")
except Exception as e:
    print(f"   ✗ エラー: {e}")
    raise

# 最初のバッチをロード（時間計測）
print("\n3. 最初のバッチをロード中...")
start = time.time()
try:
    batch = next(iter(train_loader))
    load_time = time.time() - start
    print(f"   ✓ ロード完了: {load_time:.2f}秒")
    
    # バッチの内容を確認
    print("\n4. バッチの内容:")
    for key, value in batch.items():
        if isinstance(value, torch.Tensor):
            print(f"   - {key}: shape={value.shape}, dtype={value.dtype}")
        else:
            print(f"   - {key}: {type(value)}")
    
except Exception as e:
    print(f"   ✗ エラー: {e}")
    import traceback
    traceback.print_exc()
    raise

# 診断結果
print("\n" + "=" * 60)
print("診断結果:")
print("=" * 60)
if load_time < 5:
    print(f"✓ DataLoaderは正常動作しています（{load_time:.2f}秒）")
elif load_time < 30:
    print(f"⚠ DataLoaderが遅いです（{load_time:.2f}秒）")
    print("  Google Driveからの読み込みが遅い可能性があります")
else:
    print(f"✗ DataLoaderが非常に遅いです（{load_time:.2f}秒）")
    print("  Google Driveのマウントまたはキャッシュファイルに問題がある可能性があります")
    print("  対策: データをローカル（/content/）にコピーすることを推奨")

print("\n次のステップ: 問題なければ「8. 学習の実行」に進んでください")
print("=" * 60)

## 7.6. データをローカルにコピー（高速化のため必須）

診断結果でDataLoaderが遅い場合、Google Driveからローカルストレージにデータをコピーします。
これにより学習速度が大幅に改善されます（15分 → 数秒）。

In [ ]:
from pathlib import Path
import subprocess

# Google Driveのキャッシュパス
GDRIVE_CACHE = "/content/drive/MyDrive/streamVC/data/cache"
LOCAL_DATA = "/content/data"
LOCAL_CACHE = f"{LOCAL_DATA}/cache"

print("キャッシュをローカルにコピー中...")
print(f"コピー元: {GDRIVE_CACHE}")
print(f"コピー先: {LOCAL_CACHE}")

# ファイル数をlsで取得（globより高速）
def count_files(path):
    try:
        result = subprocess.run(
            f"ls -1 '{path}' 2>/dev/null | wc -l",
            shell=True, capture_output=True, text=True
        )
        return int(result.stdout.strip())
    except:
        return 0

# Google Drive側のファイル数を取得
gdrive_train_count = count_files(f"{GDRIVE_CACHE}/libri_tts/train")
gdrive_valid_count = count_files(f"{GDRIVE_CACHE}/libri_tts/valid")
print(f"\nGoogle Drive側: train={gdrive_train_count}, valid={gdrive_valid_count}")

# ローカル側のファイル数を確認
local_train_count = count_files(f"{LOCAL_CACHE}/libri_tts/train")
local_valid_count = count_files(f"{LOCAL_CACHE}/libri_tts/valid")
print(f"ローカル側: train={local_train_count}, valid={local_valid_count}")

# trainが不足している場合はコピー
if local_train_count < gdrive_train_count:
    print(f"\ntrainデータが不足しています ({local_train_count}/{gdrive_train_count})")
    
    # tarファイルが存在するか確認（事前に作成済みなら高速）
    tar_path = "/content/drive/MyDrive/streamVC/data/cache.tar"
    if Path(tar_path).exists():
        print(f"✓ tarファイルが見つかりました: {tar_path}")
        print("tarから展開します（高速）...")
        !mkdir -p {LOCAL_DATA}
        !tar -xf {tar_path} -C {LOCAL_DATA}/
    else:
        print("tarファイルがないため、rsyncでコピーします...")
        print("(次回高速化のため、後でtarファイルを作成することを推奨)")
        !mkdir -p {LOCAL_CACHE}/libri_tts/train
        !mkdir -p {LOCAL_CACHE}/libri_tts/valid
        !mkdir -p {LOCAL_CACHE}/hubert
        !rsync -ah --progress --ignore-existing {GDRIVE_CACHE}/ {LOCAL_CACHE}/
    
    # 再カウント
    local_train_count = count_files(f"{LOCAL_CACHE}/libri_tts/train")
    local_valid_count = count_files(f"{LOCAL_CACHE}/libri_tts/valid")
    print(f"\nコピー後: train={local_train_count}, valid={local_valid_count}")
else:
    print(f"\n✓ キャッシュは既にコピー済みです")

# シンボリックリンクを更新
import shutil
if Path("data").is_symlink():
    Path("data").unlink()
elif Path("data").exists():
    shutil.rmtree("data")
Path("data").symlink_to(LOCAL_DATA)

print(f"✓ data/ -> {LOCAL_DATA} にリンク")
!du -sh {LOCAL_CACHE}

## 7.7. DataLoader動作再確認（ローカルコピー後）

ローカルにコピーした後、DataLoaderの速度を再確認します。

In [ ]:
import time
import torch
import sys
from pathlib import Path

# srcディレクトリをPythonパスに追加
sys.path.insert(0, str(Path.cwd() / "src"))

from streamvc.config import load_config
from streamvc.data.dataset import build_dataloader

print("=" * 60)
print("DataLoader速度再確認（ローカルコピー後）")
print("=" * 60)

# Config読み込み
config_path = "configs/colab_gpu_training.yaml"
cfg = load_config(config_path)

# DataLoader構築
print("\nDataLoader構築中...")
start = time.time()
train_loader = build_dataloader(cfg, split="train", batch_size=cfg.training.batch_size)
build_time = time.time() - start
print(f"✓ 構築完了: {build_time:.2f}秒")

# 最初のバッチをロード
print("\n最初のバッチをロード中...")
start = time.time()
batch = next(iter(train_loader))
load_time = time.time() - start
print(f"✓ ロード完了: {load_time:.2f}秒")

# 診断結果
print("\n" + "=" * 60)
if load_time < 5:
    print(f"✓ 高速化成功！ DataLoaderは正常動作しています（{load_time:.2f}秒）")
    print("  学習を開始できます")
elif load_time < 30:
    print(f"⚠ まだ少し遅いです（{load_time:.2f}秒）")
else:
    print(f"✗ まだ非常に遅いです（{load_time:.2f}秒）")
    print("  問題が解決していません")

print("=" * 60)

## 8. 学習の実行

### オプション1: インタラクティブ実行（ノートブック内で実行）

In [ ]:
# 学習を開始（Ctrl+Cで中断可能）
!python scripts/train.py \
    --config configs/colab_gpu_training.yaml \
    --device cuda \
    --gdrive-backup /content/drive/MyDrive/streamVC/checkpoints

### オプション2: バックグラウンド実行

In [ ]:
# バックグラウンドで学習を実行
!nohup python scripts/train.py \
    --config configs/colab_gpu_training.yaml \
    --device cuda \
    --gdrive-backup /content/drive/MyDrive/streamVC/checkpoints \
    > train.log 2>&1 &

print("Training started in background")
print("Check progress: !tail -20 train.log")

## 9. 学習の監視

In [ ]:
# ログの確認
!tail -30 train.log

In [ ]:
# TensorBoard起動
%load_ext tensorboard
%tensorboard --logdir runs/streamvc_colab_gpu/logs

## 10. Checkpointの確認

In [ ]:
# 保存されたCheckpointを確認
!ls -lh {CHECKPOINT_DIR}/

## 11. 学習の再開（セッション切断後）

In [ ]:
# 最新のCheckpointから再開
import glob

CHECKPOINT_DIR = "/content/drive/MyDrive/streamVC/checkpoints"
checkpoints = sorted(glob.glob(f"{CHECKPOINT_DIR}/step_*.pt"))
if checkpoints:
    latest_ckpt = checkpoints[-1]
    print(f"Resuming from: {latest_ckpt}")
    
    !python scripts/train.py \
        --config configs/colab_gpu_training.yaml \
        --device cuda \
        --gdrive-backup /content/drive/MyDrive/streamVC/checkpoints \
        --resume {latest_ckpt}
else:
    print("No checkpoint found. Starting from scratch.")

## Tips

### セッション切断対策
- Colabは最大12時間（無料版）または24時間（Pro）で切断されます
- Checkpointは自動的にGoogle Driveに保存されるので安全です
- 再接続後、「11. 学習の再開」から続行できます

### メモリ不足の場合
```yaml
# configs/colab_gpu_training.yaml
training:
  batch_size: 8  # 16 → 8に減らす
```

### データセットの追加
1. ローカルで新しいデータセットをダウンロード
2. `./scripts/upload_dataset_to_gdrive.sh`で追加アップロード
3. Configファイルに追加して再学習